In [2]:
import pandas as pd
import numpy as np

In [3]:
#Veri tablosunu okuma
df = pd.read_excel (r'C:\Users\samay\Downloads\Data.xlsx')

In [4]:
#Burada Notebookta görüntüleyeceğimiz tablonun tüm satır sütunlarını daraltma olmadan görebilmemiz için işlem yaptık.
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 150)

In [5]:
#Üzerinde çalışma yapacağımız sütunlardan oluşan ana veri tablomuzu oluşturuyoruz
main_df=df[['MaritalStatus','Gender','EnglishEducation','EnglishOccupation','CommuteDistance','HouseOwnerFlag','Region',\
            'YearlyIncome','TotalChildren','NumberChildrenAtHome','NumberCarsOwned','Age','BikeBuyer']]
main_df.head(5)

,MaritalStatus,Gender,EnglishEducation,EnglishOccupation,CommuteDistance,HouseOwnerFlag,Region,YearlyIncome,TotalChildren,NumberChildrenAtHome,NumberCarsOwned,Age,BikeBuyer
0,M,M,Bachelors,Professional,1-2 Miles,1,Pacific,90000,2,0,0,48,1
1,S,M,Bachelors,Professional,0-1 Miles,0,Pacific,60000,3,3,1,43,1
2,M,M,Bachelors,Professional,2-5 Miles,1,Pacific,60000,3,3,1,49,1
3,S,F,Bachelors,Professional,5-10 Miles,0,Pacific,70000,0,0,1,46,1
4,S,F,Bachelors,Professional,1-2 Miles,1,Pacific,80000,5,5,4,40,1


In [6]:
#Burada yazı tipi(object) tüm sütunların önden ve arkadan boşlukları varsa o boşlukların hepsini yok ediyoruz
main_df.columns = main_df.columns.str.strip()

In [7]:
#Burada Ev ile işyeri arasındaki uzaklığı modelin kategorik şekilde değil numerik şekilde lineer olarak dikkate almasını sağladım. 
main_df["CommuteDistanceNumeric"] = np.where(main_df["CommuteDistance"]=="0-1 Miles", 1,\
                                            (np.where(main_df["CommuteDistance"]=="1-2 Miles", 2,\
                                             np.where(main_df["CommuteDistance"]=="2-5 Miles", 4,\
                                             np.where(main_df["CommuteDistance"]=="5-10 Miles", 7,11)))))

C:\Users\samay\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


In [8]:
#Burada kategorik sütun olup sayı değerleri gösteren verilerin Yes,No şekline gelmesini istedim
main_df["HouseOwnerFlagText"] = np.where(main_df["HouseOwnerFlag"]==1, "Yes","No")

C:\Users\samay\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [9]:
main_df=main_df[['MaritalStatus','Gender','EnglishEducation','EnglishOccupation','HouseOwnerFlagText','Region',\
            'YearlyIncome','TotalChildren','NumberChildrenAtHome','NumberCarsOwned','Age','CommuteDistanceNumeric','BikeBuyer']]
main_df.head(5)

,MaritalStatus,Gender,EnglishEducation,EnglishOccupation,HouseOwnerFlagText,Region,YearlyIncome,TotalChildren,NumberChildrenAtHome,NumberCarsOwned,Age,CommuteDistanceNumeric,BikeBuyer
0,M,M,Bachelors,Professional,Yes,Pacific,90000,2,0,0,48,2,1
1,S,M,Bachelors,Professional,No,Pacific,60000,3,3,1,43,1,1
2,M,M,Bachelors,Professional,Yes,Pacific,60000,3,3,1,49,4,1
3,S,F,Bachelors,Professional,No,Pacific,70000,0,0,1,46,7,1
4,S,F,Bachelors,Professional,Yes,Pacific,80000,5,5,4,40,2,1


### Kategorik verilerin işlenmesi için object formattan integer formatına çevrilmesi gerekli.Bu noktada get_dummies metodu yardımımıza koşar

get_dummies metodunu kullandıktan sonra dummies variable trap'den kurtulmak için bir değerini sileriz.

In [10]:
marital_dummies=pd.get_dummies(main_df.MaritalStatus)
marital_dummies.head()

,M,S
0,1,0
1,0,1
2,1,0
3,0,1
4,0,1


In [11]:
merged1=pd.concat([main_df,marital_dummies],axis='columns')
merged1.head()

,MaritalStatus,Gender,EnglishEducation,EnglishOccupation,HouseOwnerFlagText,Region,YearlyIncome,TotalChildren,NumberChildrenAtHome,NumberCarsOwned,Age,CommuteDistanceNumeric,BikeBuyer,M,S
0,M,M,Bachelors,Professional,Yes,Pacific,90000,2,0,0,48,2,1,1,0
1,S,M,Bachelors,Professional,No,Pacific,60000,3,3,1,43,1,1,0,1
2,M,M,Bachelors,Professional,Yes,Pacific,60000,3,3,1,49,4,1,1,0
3,S,F,Bachelors,Professional,No,Pacific,70000,0,0,1,46,7,1,0,1
4,S,F,Bachelors,Professional,Yes,Pacific,80000,5,5,4,40,2,1,0,1


In [12]:
merged1final=merged1.drop(['MaritalStatus','S'],axis='columns')
merged1final.head()

,Gender,EnglishEducation,EnglishOccupation,HouseOwnerFlagText,Region,YearlyIncome,TotalChildren,NumberChildrenAtHome,NumberCarsOwned,Age,CommuteDistanceNumeric,BikeBuyer,M
0,M,Bachelors,Professional,Yes,Pacific,90000,2,0,0,48,2,1,1
1,M,Bachelors,Professional,No,Pacific,60000,3,3,1,43,1,1,0
2,M,Bachelors,Professional,Yes,Pacific,60000,3,3,1,49,4,1,1
3,F,Bachelors,Professional,No,Pacific,70000,0,0,1,46,7,1,0
4,F,Bachelors,Professional,Yes,Pacific,80000,5,5,4,40,2,1,0


In [13]:
gender_dummies=pd.get_dummies(main_df.Gender)
merged2=pd.concat([merged1final,gender_dummies],axis='columns')
mergedfinal2=merged2.drop(['Gender','F'],axis='columns')
mergedfinal2.head()

,EnglishEducation,EnglishOccupation,HouseOwnerFlagText,Region,YearlyIncome,TotalChildren,NumberChildrenAtHome,NumberCarsOwned,Age,CommuteDistanceNumeric,BikeBuyer,M,M
0,Bachelors,Professional,Yes,Pacific,90000,2,0,0,48,2,1,1,1
1,Bachelors,Professional,No,Pacific,60000,3,3,1,43,1,1,0,1
2,Bachelors,Professional,Yes,Pacific,60000,3,3,1,49,4,1,1,1
3,Bachelors,Professional,No,Pacific,70000,0,0,1,46,7,1,0,0
4,Bachelors,Professional,Yes,Pacific,80000,5,5,4,40,2,1,0,0


In [14]:
englisheducation_dummies=pd.get_dummies(main_df.EnglishEducation)
merged3=pd.concat([mergedfinal2,englisheducation_dummies],axis='columns')
mergedfinal3=merged3.drop(['EnglishEducation','Bachelors'],axis='columns')
mergedfinal3.head()

,EnglishOccupation,HouseOwnerFlagText,Region,YearlyIncome,TotalChildren,NumberChildrenAtHome,NumberCarsOwned,Age,CommuteDistanceNumeric,BikeBuyer,M,M,Graduate Degree,High School,Partial College,Partial High School
0,Professional,Yes,Pacific,90000,2,0,0,48,2,1,1,1,0,0,0,0
1,Professional,No,Pacific,60000,3,3,1,43,1,1,0,1,0,0,0,0
2,Professional,Yes,Pacific,60000,3,3,1,49,4,1,1,1,0,0,0,0
3,Professional,No,Pacific,70000,0,0,1,46,7,1,0,0,0,0,0,0
4,Professional,Yes,Pacific,80000,5,5,4,40,2,1,0,0,0,0,0,0


In [15]:
englishoccupation_dummies=pd.get_dummies(main_df.EnglishOccupation)
merged4=pd.concat([mergedfinal3,englishoccupation_dummies],axis='columns')
mergedfinal4=merged4.drop(['EnglishOccupation','Professional'],axis='columns')
mergedfinal4.head()

,HouseOwnerFlagText,Region,YearlyIncome,TotalChildren,NumberChildrenAtHome,NumberCarsOwned,Age,CommuteDistanceNumeric,BikeBuyer,M,M,Graduate Degree,High School,Partial College,Partial High School,Clerical,Management,Manual,Skilled Manual
0,Yes,Pacific,90000,2,0,0,48,2,1,1,1,0,0,0,0,0,0,0,0
1,No,Pacific,60000,3,3,1,43,1,1,0,1,0,0,0,0,0,0,0,0
2,Yes,Pacific,60000,3,3,1,49,4,1,1,1,0,0,0,0,0,0,0,0
3,No,Pacific,70000,0,0,1,46,7,1,0,0,0,0,0,0,0,0,0,0
4,Yes,Pacific,80000,5,5,4,40,2,1,0,0,0,0,0,0,0,0,0,0


In [16]:
houseownerflagtext_dummies=pd.get_dummies(main_df.HouseOwnerFlagText)
merged5=pd.concat([mergedfinal4,houseownerflagtext_dummies],axis='columns')
mergedfinal5=merged5.drop(['HouseOwnerFlagText','No'],axis='columns')
mergedfinal5.head()

,Region,YearlyIncome,TotalChildren,NumberChildrenAtHome,NumberCarsOwned,Age,CommuteDistanceNumeric,BikeBuyer,M,M,Graduate Degree,High School,Partial College,Partial High School,Clerical,Management,Manual,Skilled Manual,Yes
0,Pacific,90000,2,0,0,48,2,1,1,1,0,0,0,0,0,0,0,0,1
1,Pacific,60000,3,3,1,43,1,1,0,1,0,0,0,0,0,0,0,0,0
2,Pacific,60000,3,3,1,49,4,1,1,1,0,0,0,0,0,0,0,0,1
3,Pacific,70000,0,0,1,46,7,1,0,0,0,0,0,0,0,0,0,0,0
4,Pacific,80000,5,5,4,40,2,1,0,0,0,0,0,0,0,0,0,0,1


In [17]:
region_dummies=pd.get_dummies(main_df.Region)
merged6=pd.concat([mergedfinal5,region_dummies],axis='columns')
mergedfinal6=merged6.drop(['Region','Pacific'],axis='columns')
mergedfinal6.head()
#İlk M marital Status ikincisi Male

,YearlyIncome,TotalChildren,NumberChildrenAtHome,NumberCarsOwned,Age,CommuteDistanceNumeric,BikeBuyer,M,M,Graduate Degree,High School,Partial College,Partial High School,Clerical,Management,Manual,Skilled Manual,Yes,Europe,North America
0,90000,2,0,0,48,2,1,1,1,0,0,0,0,0,0,0,0,1,0,0
1,60000,3,3,1,43,1,1,0,1,0,0,0,0,0,0,0,0,0,0,0
2,60000,3,3,1,49,4,1,1,1,0,0,0,0,0,0,0,0,1,0,0
3,70000,0,0,1,46,7,1,0,0,0,0,0,0,0,0,0,0,0,0,0
4,80000,5,5,4,40,2,1,0,0,0,0,0,0,0,0,0,0,1,0,0


-------

### TRAIN/TEST SPLIT YÖNTEMİ

In [25]:
X=mergedfinal6.drop('BikeBuyer',axis=1)
y=mergedfinal6['BikeBuyer']

In [30]:
from sklearn.model_selection import train_test_split

In [31]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.3,random_state=1)

In [32]:
from sklearn.linear_model import LogisticRegression

In [59]:
logmodel=LogisticRegression(solver='newton-cg')
logmodel.fit(X_train,y_train)

C:\Users\samay\Anaconda3\lib\site-packages\scipy\optimize\linesearch.py:462: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
C:\Users\samay\Anaconda3\lib\site-packages\scipy\optimize\linesearch.py:313: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
C:\Users\samay\Anaconda3\lib\site-packages\scipy\optimize\linesearch.py:462: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
C:\Users\samay\Anaconda3\lib\site-packages\scipy\optimize\linesearch.py:313: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
C:\Users\samay\Anaconda3\lib\site-packages\scipy\optimize\linesearch.py:462: LineSearchWarning: The line search algorithm did not converge
  warn('The line 

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='newton-cg',
          tol=0.0001, verbose=0, warm_start=False)

In [60]:
predictions=logmodel.predict(X_test)

In [61]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,predictions)

0.644969347277317

----

### StandardScale ile standardizasyon yapıp regresyon yapma testi

In [67]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.3,random_state=0)

In [74]:
from sklearn.preprocessing import StandardScaler
sc=StandardScaler()
X_train=sc.fit_transform(X_train)
X_test=sc.transform(X_test)

C:\Users\samay\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:645: DataConversionWarning: Data with input dtype uint8, int32, int64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
C:\Users\samay\Anaconda3\lib\site-packages\sklearn\base.py:464: DataConversionWarning: Data with input dtype uint8, int32, int64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)
C:\Users\samay\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: DataConversionWarning: Data with input dtype uint8, int32, int64 were all converted to float64 by StandardScaler.
  after removing the cwd from sys.path.


In [75]:
classifier=LogisticRegression(random_state=0)
classifier.fit(X_train,y_train)

C:\Users\samay\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=0, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

In [76]:
y_pred=classifier.predict(X_test)

In [77]:
accuracy_score(y_test,y_pred)

0.641002524341868

0.003 azaldı

-----

### Train/Test yapmadan accuracy sonucunu görme

In [82]:
model=LogisticRegression(solver='newton-cg')

In [83]:
model.fit(X,y)

C:\Users\samay\Anaconda3\lib\site-packages\scipy\optimize\linesearch.py:462: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
C:\Users\samay\Anaconda3\lib\site-packages\scipy\optimize\linesearch.py:313: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
C:\Users\samay\Anaconda3\lib\site-packages\scipy\optimize\linesearch.py:462: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
C:\Users\samay\Anaconda3\lib\site-packages\scipy\optimize\linesearch.py:313: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
C:\Users\samay\Anaconda3\lib\site-packages\scipy\optimize\linesearch.py:462: LineSearchWarning: The line search algorithm did not converge
  warn('The line 

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='newton-cg',
          tol=0.0001, verbose=0, warm_start=False)

In [84]:
model.score(X,y)

0.6426098247132656

0.002 azaldı

----

### Yaptığımız feature importance ve Univariate Selection yönteminden çıkardığımız sonuca göre Gender,marital status,houseownerflag sütunlarını çıkarırsak modelimizde accuracy artışı görebilirdik bunu train/test modelinde de göreceğiz

In [17]:
mergedfinal8=mergedfinal6.drop(['M','Yes'],axis='columns')

In [18]:
mergedfinal8.head()

,YearlyIncome,TotalChildren,NumberChildrenAtHome,NumberCarsOwned,Age,CommuteDistanceNumeric,BikeBuyer,Graduate Degree,High School,Partial College,Partial High School,Clerical,Management,Manual,Skilled Manual,Europe,North America
0,90000,2,0,0,48,2,1,0,0,0,0,0,0,0,0,0,0
1,60000,3,3,1,43,1,1,0,0,0,0,0,0,0,0,0,0
2,60000,3,3,1,49,4,1,0,0,0,0,0,0,0,0,0,0
3,70000,0,0,1,46,7,1,0,0,0,0,0,0,0,0,0,0
4,80000,5,5,4,40,2,1,0,0,0,0,0,0,0,0,0,0


In [89]:
X=mergedfinal8.drop('BikeBuyer',axis=1)
y=mergedfinal8['BikeBuyer']

In [90]:
from sklearn.model_selection import train_test_split

In [91]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.3,random_state=1)

In [92]:
from sklearn.linear_model import LogisticRegression

In [93]:
logmodel=LogisticRegression(solver='newton-cg')
logmodel.fit(X_train,y_train)

C:\Users\samay\Anaconda3\lib\site-packages\scipy\optimize\linesearch.py:462: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
C:\Users\samay\Anaconda3\lib\site-packages\scipy\optimize\linesearch.py:313: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
C:\Users\samay\Anaconda3\lib\site-packages\scipy\optimize\linesearch.py:462: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
C:\Users\samay\Anaconda3\lib\site-packages\scipy\optimize\linesearch.py:313: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
C:\Users\samay\Anaconda3\lib\site-packages\scipy\optimize\linesearch.py:462: LineSearchWarning: The line search algorithm did not converge
  warn('The line 

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='newton-cg',
          tol=0.0001, verbose=0, warm_start=False)

In [94]:
predictions=logmodel.predict(X_test)

In [95]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,predictions)

0.6458708979444645

3 sütunu çıkardığımızda accuracy oranında 0.001'lik artış oldu

### Şimdi de sonucu en fazla etkileyen iki sütun olan 'Age' ve 'YearlyIncome' sütunların veri setine dahil edilmediğindeki sonucu görelim

In [19]:
mergedfinal9=mergedfinal6.drop(['Age','YearlyIncome'],axis='columns')

In [20]:
mergedfinal9.head()

,TotalChildren,NumberChildrenAtHome,NumberCarsOwned,CommuteDistanceNumeric,BikeBuyer,M,M,Graduate Degree,High School,Partial College,Partial High School,Clerical,Management,Manual,Skilled Manual,Yes,Europe,North America
0,2,0,0,2,1,1,1,0,0,0,0,0,0,0,0,1,0,0
1,3,3,1,1,1,0,1,0,0,0,0,0,0,0,0,0,0,0
2,3,3,1,4,1,1,1,0,0,0,0,0,0,0,0,1,0,0
3,0,0,1,7,1,0,0,0,0,0,0,0,0,0,0,0,0,0
4,5,5,4,2,1,0,0,0,0,0,0,0,0,0,0,1,0,0


In [98]:
X=mergedfinal9.drop('BikeBuyer',axis=1)
y=mergedfinal9['BikeBuyer']

In [33]:
from sklearn.model_selection import train_test_split

In [100]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.3,random_state=1)

In [101]:
from sklearn.linear_model import LogisticRegression

In [102]:
logmodel=LogisticRegression(solver='newton-cg')
logmodel.fit(X_train,y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='newton-cg',
          tol=0.0001, verbose=0, warm_start=False)

In [103]:
predictions=logmodel.predict(X_test)

In [104]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,predictions)

0.6345113595384061

Burada sonuç normalden 0.01 azaldı

-----

### Sadece regresyon katsayısı 0'dan büyük olan sütunları eğittiğimizdeki sonuç

In [18]:
X=mergedfinal6.drop('BikeBuyer',axis=1)
y=mergedfinal6['BikeBuyer']

In [141]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.3,random_state=0)

In [142]:
logreg=LogisticRegression(fit_intercept=False)
logreg.fit(X_train,y_train)

C:\Users\samay\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=False,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

In [144]:
np.round(logreg.coef_,decimals=2)>0

array([[False, False, False, False,  True, False, False, False, False,
         True,  True, False,  True, False,  True,  True,  True, False,
        False]])

In [114]:
X.columns

Index(['YearlyIncome', 'TotalChildren', 'NumberChildrenAtHome', 'NumberCarsOwned', 'Age', 'CommuteDistanceNumeric', 'M', 'M', 'Graduate Degree',
       'High School', 'Partial College', 'Partial High School', 'Clerical', 'Management', 'Manual', 'Skilled Manual', 'Yes', 'Europe',
       'North America'],
      dtype='object')

In [116]:
logreg='Age','High School','Partial College','Clerical','Manual','Skilled Manual','Yes'

In [128]:
X=mergedfinal6[['Age','High School','Partial College','Clerical','Manual','Skilled Manual','Yes']]
y=mergedfinal6['BikeBuyer']

In [129]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.3,random_state=1)

In [130]:
from sklearn.linear_model import LogisticRegression

In [134]:
logmodel=LogisticRegression()
logmodel.fit(X_train,y_train)

C:\Users\samay\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

In [135]:
predictions=logmodel.predict(X_test)

In [136]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,predictions)

0.5580598629642985

0.1 azaldı

----

### GridSearch yöntemi kullanarak Lojistik Regresyon modelimizdeki en optimal C katsayısını bulma

In [198]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler(feature_range = (0,1))

scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

C:\Users\samay\Anaconda3\lib\site-packages\sklearn\utils\validation.py:595: DataConversionWarning: Data with input dtype int64 was converted to float64 by MinMaxScaler.
  warnings.warn(msg, DataConversionWarning)


In [197]:
X=mergedfinal6.drop('BikeBuyer',axis=1).values
y=mergedfinal6['BikeBuyer'].values

In [34]:
X_train,X_test,y_train,y_test  = train_test_split(X,y,test_size=0.3,random_state=3)

In [200]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
grid={"C":np.logspace(-3,6,7), "penalty":["l1","l2"]}# l1 lasso l2 ridge
logreg=LogisticRegression()
logreg_cv=GridSearchCV(logreg,grid,cv=10)
logreg_cv.fit(X_train,y_train)

C:\Users\samay\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\samay\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\samay\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\samay\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\samay\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence t

C:\Users\samay\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\samay\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\samay\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\samay\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\samay\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence t

C:\Users\samay\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\samay\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\samay\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\samay\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\samay\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence t

C:\Users\samay\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\samay\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\samay\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\samay\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\samay\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence t

GridSearchCV(cv=10, error_score='raise-deprecating',
       estimator=LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'C': array([1.00000e-03, 3.16228e-02, 1.00000e+00, 3.16228e+01, 1.00000e+03,
       3.16228e+04, 1.00000e+06]), 'penalty': ['l1', 'l2']},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [201]:
print("tuned hpyerparameters :(best parameters) ",logreg_cv.best_params_)
print("accuracy :",logreg_cv.best_score_)

tuned hpyerparameters :(best parameters)  {'C': 1.0, 'penalty': 'l2'}
accuracy : 0.6441490183954244


In [202]:
logreg2=LogisticRegression(C=9,penalty="l2")
logreg2.fit(X_train,y_train)
print("score",logreg2.score(X_test,y_test))

C:\Users\samay\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


score 0.6393797331410025


### RandomForestClassifier modeli kullanarak accuracy score'u arttırma

RandomForestClassifier modelinin en iyi parametrelerini bulmak için tek tek deneme yanılma yoluyla işlem yapamayız.
Bu yüzden GridSearch modelini kullandık bulduğumuz parametreyi modele uyguladık.

In [203]:
from sklearn.pipeline import *

In [204]:
from sklearn.ensemble import RandomForestClassifier

In [205]:
# Create first pipeline for base without reducing features.

pipe = Pipeline([('classifier' , RandomForestClassifier())])
# pipe = Pipeline([('classifier', RandomForestClassifier())])




In [214]:
# Create param grid.

param_grid = [
    {'classifier' : [LogisticRegression()],
     'classifier__penalty' : ['l1', 'l2'],
    'classifier__C' : np.logspace(-4, 4, 20),
    'classifier__solver' : ['liblinear']},
    {'classifier' : [RandomForestClassifier()],
    'classifier__n_estimators' : list(range(10,101,10))}
]


In [215]:
clf = GridSearchCV(pipe, param_grid = param_grid, cv = 5, verbose=True, n_jobs=-1)

# Fit on data

best_clf = clf.fit(X_train, y_train)

Fitting 5 folds for each of 50 candidates, totalling 250 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:    4.4s
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed:   14.4s
[Parallel(n_jobs=-1)]: Done 250 out of 250 | elapsed:   30.8s finished


In [221]:
best_clf.best_params_

{'classifier': RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
             max_depth=None, max_features='auto', max_leaf_nodes=None,
             min_impurity_decrease=0.0, min_impurity_split=None,
             min_samples_leaf=1, min_samples_split=2,
             min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=None,
             oob_score=False, random_state=None, verbose=0,
             warm_start=False), 'classifier__n_estimators': 100}

In [222]:
rfc=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
             max_depth=None, max_features='auto', max_leaf_nodes=None,
             min_impurity_decrease=0.0, min_impurity_split=None,
             min_samples_leaf=1, min_samples_split=2,
             min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=None,
             oob_score=False, random_state=None, verbose=0,
             warm_start=False)

In [223]:
rfc.fit(X_train,y_train)
print("score",rfc.score(X_test,y_test))

score 0.7908402452217814


**Accuracy score'u yüzde 64ten yüzde 79'a çıkarmayı başardık**

In [226]:
Xdf = pd.DataFrame(X)

In [227]:
X_new=Xdf.sample(5,random_state=99)
X_new

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18
5471,10000,4,0,2,101,1,1,0,0,1,0,0,0,0,1,0,0,1,0
12364,40000,0,0,1,50,4,0,0,0,0,0,0,0,0,0,0,0,0,1
7989,70000,5,0,2,63,4,1,0,0,0,0,0,0,1,0,0,1,0,1
10396,40000,0,0,2,38,2,0,0,0,0,1,0,0,0,0,1,0,0,1
758,70000,5,5,3,40,11,1,1,0,0,0,0,0,0,0,0,1,0,0


In [233]:
mergedfinal6.columns

Index(['YearlyIncome', 'TotalChildren', 'NumberChildrenAtHome', 'NumberCarsOwned', 'Age', 'CommuteDistanceNumeric', 'BikeBuyer', 'M', 'M',
       'Graduate Degree', 'High School', 'Partial College', 'Partial High School', 'Clerical', 'Management', 'Manual', 'Skilled Manual', 'Yes',
       'Europe', 'North America'],
      dtype='object')

In [228]:
rfc.predict(X_new)

array([0, 0, 0, 1, 0], dtype=int64)

Burada da yeni gelen müşterilerin bilgilerini X_new vasıtasıyla modele soktuk.Daha sonra predict edip alıp almayacaklarını gördük.

In [28]:
mergedfinal6.iloc[5471,:]

YearlyIncome              10000
TotalChildren                 4
NumberChildrenAtHome          0
NumberCarsOwned               2
Age                         101
CommuteDistanceNumeric        1
BikeBuyer                     0
M                             1
M                             0
Graduate Degree               0
High School                   1
Partial College               0
Partial High School           0
Clerical                      0
Management                    0
Manual                        1
Skilled Manual                0
Yes                           0
Europe                        1
North America                 0
Name: 5471, dtype: int64

In [24]:
mergedfinal6.iloc[12364,6]

0

In [25]:
mergedfinal6.iloc[7989,6]

0

In [26]:
mergedfinal6.iloc[10396,6]

1

In [27]:
mergedfinal6.iloc[758,6]

0

**Veri setindeki gerçek değerlerle geliştirdiğimiz modelimizin sonucunu kıyasladığımızda modelimiz tüm müşterilerin satın alma davranışını doğru tahmin etmiştir**